In [10]:
import yaml

with open("ollama_config.yaml", "r") as file:
    data = yaml.safe_load(file)

print("Loaded YAML Data:")
data

Loaded YAML Data:


{'vectordb': [{'name': 'chroma_mpnet',
   'db_type': 'chroma',
   'client_type': 'persistent',
   'collection_name': 'huggingface_all_mpnet_base_v2',
   'embedding_model': 'huggingface_all_mpnet_base_v2',
   'path': '/home/lyb/RAG/experiments/chroma_mpnet'}],
 'node_lines': [{'node_line_name': 'retrieve_node_line',
   'nodes': [{'node_type': 'retrieval',
     'strategy': {'metrics': ['retrieval_f1',
       'retrieval_recall',
       'retrieval_precision']},
     'top_k': 3,
     'modules': [{'module_type': 'bm25'},
      {'module_type': 'vectordb', 'vectordb': 'chroma_mpnet'},
      {'module_type': 'hybrid_rrf',
       'target_modules': "('bm25', 'vectordb')",
       'rrf_k': [3, 5, 10]},
      {'module_type': 'hybrid_cc',
       'target_modules': "('bm25', 'vectordb')",
       'weights': '(0.5, 0.5)'}]}]},
  {'node_line_name': 'post_retrieve_node_line',
   'nodes': [{'node_type': 'prompt_maker',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']},
     'modules': [{'module

In [12]:
test_modules = [{'module_type': 'HybridCC',
                            'target_module_params': [{'top_k': 3, 'vectordb': 'chroma_mpnet'}, {'top_k': 3}],
                            'target_modules': ['VectorDB', 'BM25'],
                            'weights': [0.3, 0.7]},
                {'module_type': 'hybrid_cc',
                    'target_modules': "('bm25', 'vectordb')",
                    'target_module_params': [{'top_k': 3}, {'top_k': 3, 'vectordb': 'chroma_mpnet'}],
                    'weights': ['(0.5, 0.5)']},
                {'module_type': 'hybrid_cc',
                    'target_modules': "('bm25', 'vectordb')",
                    'target_module_params': [{'top_k': 3}, {'top_k': 3, 'vectordb': 'chroma_mpnet'}],
                    'weights': ['(0.7, 0.3)']},
                {'module_type': 'bm25'},
                {'module_type': 'vectordb', 'vectordb': 'chroma_mpnet'},
                {'module_type': 'hybrid_rrf',
                    'target_modules': "('bm25', 'vectordb')",
                    'target_module_params': [{'top_k': 3}, {'top_k': 3, 'vectordb': 'chroma_mpnet'}],
                    'rrf_k': [5]},
                {'module_type': 'hybrid_rrf',
                    'target_modules': "('bm25', 'vectordb')",
                    'target_module_params': [{'top_k': 3}, {'top_k': 3, 'vectordb': 'chroma_mpnet'}],
                    'rrf_k': [3]},
                {'module_type': 'hybrid_rrf',
                    'target_modules': "('bm25', 'vectordb')",
                    'target_module_params': [{'top_k': 3}, {'top_k': 3, 'vectordb': 'chroma_mpnet'}],
                    'rrf_k': [10]},]

for i, module in enumerate(test_modules):
    for node in data['node_lines']:
        if node['node_line_name'] == 'retrieve_node_line':
            for node_line in node['nodes']:
                node_line['modules'] = [module]

            # Save the modified data back to a YAML file
            with open(f"./candidate_config/ollama_config_{i}.yaml", "w") as file:
                yaml.safe_dump(data, file, default_flow_style=False, sort_keys=False)

In [8]:
with open("eli5/0/best.yaml", "r") as file:
    tmp = yaml.safe_load(file)

tmp

{'node_lines': [{'node_line_name': 'retrieve_node_line',
   'nodes': [{'modules': [{'module_type': 'HybridCC',
       'target_module_params': [{'top_k': 3, 'vectordb': 'chroma_mpnet'},
        {'top_k': 3}],
       'target_modules': ['VectorDB', 'BM25'],
       'top_k': 3,
       'weight': 0.0,
       'weights': [0.3, 0.7]}],
     'node_type': 'retrieval',
     'strategy': {'metrics': ['retrieval_f1',
       'retrieval_recall',
       'retrieval_precision']}}]},
  {'node_line_name': 'post_retrieve_node_line',
   'nodes': [{'modules': [{'module_type': 'Fstring',
       'prompt': 'Read the passages and answer the given question. \n Question: {query} \n Passage: {retrieved_contents} \n Answer : '}],
     'node_type': 'prompt_maker',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']}},
    {'modules': [{'batch': 1,
       'llm': 'ollama',
       'model': 'llama3',
       'module_type': 'LlamaIndexLLM',
       'temperature': 0.5}],
     'node_type': 'generator',
     'strategy'